In [ ]:
import openforcefield as off
from rdkit import Chem
from rdkit.Chem import AllChem
from simtk import openmm, unit
from simtk.openmm import app
from openforcefield.topology import Topology
from openforcefield.topology import Molecule
from openforcefield.typing.engines.smirnoff import ForceField
from rdkit.Chem.Draw import IPythonConsole

from simtk.openmm.openmm import Platform


In [ ]:
import openmmtools
pts = openmmtools.utils.get_available_platforms()

for plat in pts:
    print(plat.getName())
    
openmmtools.utils.get_fastest_platform().getName()

In [ ]:
ff = ForceField('test_forcefields/smirnoff99Frosst.offxml')

In [ ]:
%time rdmol = Chem.MolFromSmiles('CCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC')
%time rdmol = Chem.AddHs(rdmol)
%time AllChem.EmbedMolecule(rdmol)
# Chem.rdmolops.AssignAtomChiralTagsFromStructure(rdmol)
%time ofmol = Molecule.from_rdkit(rdmol)

# ofmol.generate_conformers(n_conformers=10)
%time topology = ofmol.to_topology()

%time org_system = ff.create_openmm_system(topology)
# pos = ofmol.conformers[0]



In [ ]:
pos = ofmol.conformers[0]

In [ ]:
def get_energy(system, positions, platform):
    integrator = openmm.VerletIntegrator(1.0 * unit.femtoseconds)
    context = openmm.Context(system, integrator, platform)
    context.setPositions(positions)
    state = context.getState(getEnergy=True)
    energy = state.getPotentialEnergy().in_units_of(unit.kilocalories_per_mole)
    return energy

rdmol = Chem.MolFromSmiles('CCCCCCCCCCCCCCCC')
rdmol = Chem.AddHs(rdmol)
AllChem.EmbedMolecule(rdmol)
# Chem.rdmolops.AssignAtomChiralTagsFromStructure(rdmol)
ofmol = Molecule.from_rdkit(rdmol)

# ofmol.generate_conformers(n_conformers=10)
topology = ofmol.to_topology()

org_system = ff.create_openmm_system(topology)
pos = ofmol.conformers[0]

In [ ]:
for plat in openmmtools.utils.get_available_platforms():
    print(plat.getName())
    %timeit get_energy(org_system, pos, plat)

In [ ]:
from simtk.openmm.openmm import Platform

new_system = ff.create_openmm_system(topology)
# new_energy = get_energy(new_system, pos)
from sys import stdout
def minimizeOpenMM(Topology, System, Positions, platform):
    
    platform = Platform.getPlatformByName('CPU')
#     properties = {'DeviceIndex': '0', 'Precision': 'double'}
    integrator = openmm.LangevinIntegrator(
                                        300.0 * unit.kelvin,
                                        1.0 / unit.picosecond,
                                        2.0 * unit.femtosecond)
                                        #.002 * unit.picoseconds)
    simulation = app.Simulation(Topology, System, integrator, platform)
    simulation.context.setPositions(Positions)
    simulation.minimizeEnergy(tolerance=5.0E-9, maxIterations=2000)
#     state =  simulation.context.getState(getPositions=True, getEnergy=True)
#     positions =state.getPositions(asNumpy=True)
#     energy = state.getPotentialEnergy().in_units_of(unit.kilocalories_per_mole)
     
#     simulation.reporters.append(app.StateDataReporter(stdout, 1000, step=True, potentialEnergy=True, temperature=True))
#     simulation.step(30000)
#     print(energy)
#     positions = positions / unit.angstroms
#     coordlist = list()
#     for atom_coords in positions:
#         coordlist += [i for i in atom_coords]
#     return coordlist, positions
    return None, None

for plat in openmmtools.utils.get_available_platforms():
    print(plat.getName())
    ofmol.generate_conformers(n_conformers=1)
    topology = ofmol.to_topology()

    org_system = ff.create_openmm_system(topology)
    pos = ofmol.conformers[0]
    %time _, _=minimizeOpenMM(topology, org_system, pos, plat)

In [1]:
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import Draw
from rdkit.Chem.Draw import IPythonConsole
import py3Dmol
from simtk.openmm import openmm

from openforcefield.topology import Molecule

# Define the keyword arguments to feed to ForceField
from simtk import unit
from simtk.openmm import app

In [2]:
from rdkit import Chem
from rdkit.Chem.rdmolfiles import SDWriter
# molOrig = Chem.MolFromSmiles("COc1cc(C2Oc3c(OC)cc(C=CCO)cc3C2CO)ccc1[O]")
molOrig = Chem.MolFromSmiles('COc1cc(C(O)C(CO)Oc2c(OC)cc(C(O)C(CO)Oc3ccc(C4Oc5c(OC)cc(C6Oc7c(OC)cc(C8Oc9c(OC)cc(C=CCO)cc9C8CO)cc7C6CO)cc5C4CO)cc3OC)cc2-c2cc(C3OCC4C(c5ccc(O)c(OC)c5)OCC34)cc(OC)c2O)ccc1O')
molOrig = Chem.AddHs(molOrig)
res = Chem.AllChem.EmbedMolecule(molOrig)
Chem.AllChem.MMFFOptimizeMoleculeConfs(molOrig)


Chem.rdmolops.GetFormalCharge(molOrig)

# writer = SDWriter('out.sdf')
# for mol in [molOrig]:
#     writer.write(mol)

0

In [ ]:
# AllChem.EmbedMolecule(molOrig)
# Chem.rdmolops.AssignAtomChiralTagsFromStructure(molOrig)
from openforcefield.topology import Molecule
ofmol = Molecule.from_rdkit(molOrig, allow_undefined_stereo=True)
# ofmol = Molecule.from_smiles('COc1cc(C2Oc3c(OC)cc(C=CCO)cc3C2CO)ccc1[O]', allow_undefined_stereo=True)
# ofmol = Molecule.from_smiles(ofmol.to_smiles(), allow_undefined_stereo=True)

#generate_unique_atom_names(ofmol)
# %time ofmol.generate_conformers(n_conformers=1)
pos = ofmol.conformers[0]
ofmol.name = 'molecule'

from simtk.openmm import app
from simtk import unit
forcefield_kwargs = { 'constraints' : app.HBonds, 'rigidWater' : True, 'removeCMMotion' : False, 'hydrogenMass' : 4*unit.amu }
# Initialize a SystemGenerator using GAFF
from openmmforcefields.generators import SystemGenerator
system_generator = SystemGenerator(small_molecule_forcefield='gaff-2.11', molecules=[ofmol], forcefield_kwargs=forcefield_kwargs, cache='db10.json')
# Create an OpenMM System from an OpenMM Topology object
%time system = system_generator.create_system(ofmol.to_topology().to_openmm())
# takes nearly 2 minutes wall time on first go, have to ask why

Warning (not error because allow_undefined_stereo=True): 
Warning (not error because allow_undefined_stereo=True): 


Checking against [H][O][C]1=[C]([H])[C]([H])=[C]([C]2([H])[O][C]3=[C]([O][C]([H])([H])[H])[C]([H])=[C]([C]([H])=[C]([H])[C]([H])([H])[O][H])[C]([H])=[C]3[C]2([H])[C]([H])([H])[O][H])[C]([H])=[C]1[O][C]([H])([H])[H]


Warning (not error because allow_undefined_stereo=True): 


In [8]:
def get_energy(system, positions, platform):
    integrator = openmm.VerletIntegrator(1.0 * unit.femtoseconds)
    context = openmm.Context(system, integrator, platform)
    context.setPositions(positions)
    state = context.getState(getEnergy=True)
    energy = state.getPotentialEnergy().in_units_of(unit.kilocalories_per_mole)
    return energy


import openmmtools
pts = openmmtools.utils.get_available_platforms()

for plat in pts:
    print(plat.getName())
    %time get_energy(system, pos, plat)

Reference
CPU times: user 2.28 ms, sys: 1.09 ms, total: 3.37 ms
Wall time: 2.11 ms
CPU
CPU times: user 2.86 ms, sys: 6.62 ms, total: 9.48 ms
Wall time: 5 ms
CUDA
CPU times: user 25.5 ms, sys: 145 ms, total: 171 ms
Wall time: 208 ms
OpenCL
CPU times: user 45.1 ms, sys: 131 ms, total: 176 ms
Wall time: 256 ms
